In [ ]:
# Instaliraj potrebne pakete (pokreni samo jednom)
!pip install -U anthropic google-generativeai llama-index-llms-anthropic llama-index-embeddings-huggingface llama-index-llms-ollama llama-index-llms-gemini llama-index-embeddings-gemini
!pip install -U llama-index-llms-google-genai llama-index-embeddings-google-genai

import os
import json
import time
import pandas as pd
from datetime import datetime
from typing import List, Dict, Any
from dotenv import load_dotenv

import openai
import anthropic
import google.generativeai as genai # Za Gemini Judge

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings, StorageContext, load_index_from_storage
from llama_index.core.node_parser import SentenceSplitter, SemanticSplitterNodeParser

# OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI as LlamaOpenAI 

# Gemini (Google)
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding

# Ostalo (Lokalno/Anthropic ako zatreba)
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

load_dotenv()

# Konfiguracija Google API-ja za Judge funkciju
if os.getenv("GOOGLE_API_KEY"):
    genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

print("Sustav spreman. Svi moduli (OpenAI, Gemini, Lokalni) su učitani.")

  Attempting uninstall: anthropic
    Found existing installation: anthropic 0.83.0
    Uninstalling anthropic-0.83.0:
      Successfully uninstalled anthropic-0.83.0



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Sustav spreman. Svi moduli (OpenAI, Gemini, Lokalni) su učitani.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
class RAGConfig:
    # API Ključevi
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    
    if not OPENAI_API_KEY:
        print("UPOZORENJE: OPENAI_API_KEY nedostaje!")
    if not GOOGLE_API_KEY:
        print("UPOZORENJE: GOOGLE_API_KEY nedostaje!")
    
    # Putanje
    RESULTS_DIR = "./Rezultati" 
    STORAGE_DIR = "./Storage"  # Za spremanje indeksa na disk (OpenAI i Gemini odvojeno)
    
    DATASETS = {
        "Dataset_1": {
            "pdf_dir": "./Dokumenti/Fiskalizacija",        # Mapa s PDF-ovima za prvi dataset
            "qa_path": "./eq_fiskalizacija.json"      # JSON s pitanjima za prvi dataset
        },
        "Dataset_2": {
            "pdf_dir": "./Dokumenti/EU/",        # Mapa s PDF-ovima za drugi dataset
            "qa_path": "./eq_eu_treaty.json"      # JSON s pitanjima za drugi dataset
        }
    }

    
    # OpenAI Modeli
    LLM_MODEL = "gpt-4o-mini"
    EMBEDDING_MODEL = "text-embedding-3-large"
    
    # Google Gemini Modeli
    GEMINI_LLM_MODEL = "models/gemini-2.5-flash"    # Brz i jeftin za generiranje
    GEMINI_EMBED_MODEL = "models/gemini-embedding-001" # Googleov najnoviji embedding
    GEMINI_EVAL_MODEL = "models/gemini-2.5-flash"      # Pametniji model za suca (Judge)
    
    SLEEP_BETWEEN_QUERIES = 0.5 
    
    CHUNKING_STRATEGIES = {
        "micro_overlap20": {
            "type": "fixed",
            "chunk_size": 128,
            "chunk_overlap": 20
        },
        "standard_overlap50": {
            "type": "fixed",
            "chunk_size": 512,
            "chunk_overlap": 50
        },
        "macro_overlap100": {
            "type": "fixed",
            "chunk_size": 1024,
            "chunk_overlap": 100
        },
        "semantic": {
            "type": "semantic",
            "buffer_size": 1,
            "breakpoint_percentile_threshold": 95
        }
    }
    

os.makedirs(RAGConfig.RESULTS_DIR, exist_ok=True)
os.makedirs(RAGConfig.STORAGE_DIR, exist_ok=True)

print(f"Konfiguracija učitana. Rezultati će biti spremljeni u {RAGConfig.RESULTS_DIR}")

Konfiguracija učitana. Rezultati će biti spremljeni u ./Rezultati


In [ ]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbedding

embed_model_openai = OpenAIEmbedding(
    model_name=RAGConfig.EMBEDDING_MODEL, 
    api_key=RAGConfig.OPENAI_API_KEY
)

llm_openai = LlamaOpenAI(
    model=RAGConfig.LLM_MODEL, 
    temperature=0.1, 
    api_key=RAGConfig.OPENAI_API_KEY
)

print("Inicijaliziram Gemini modele (Stabilna verzija)...")

embed_model_gemini = GeminiEmbedding(
    model_name=RAGConfig.GEMINI_EMBED_MODEL,
    api_key=RAGConfig.GOOGLE_API_KEY
)

llm_gemini = Gemini(
    model=RAGConfig.GEMINI_LLM_MODEL,
    api_key=RAGConfig.GOOGLE_API_KEY,
    temperature=0.1
)

Settings.embed_model = embed_model_openai
Settings.llm = llm_openai

print("\n[OK] Svi modeli inicijalizirani bez greške i bez upozorenja!")
print(f"[INFO] Default LLM: {Settings.llm.model}")
print(f"[INFO] Gemini LLM spreman: {llm_gemini.model}")

Inicijaliziram Gemini modele (Stabilna verzija)...

[OK] Svi modeli inicijalizirani bez greške i bez upozorenja!
[INFO] Default LLM: gpt-4o-mini
[INFO] Gemini LLM spreman: models/gemini-2.5-flash


In [ ]:
def get_or_build_index(dataset_name, strategy_name, strategy_config, embed_model, suffix, documents):
    folder_name = f"{dataset_name}_{strategy_name}_{suffix}"
    persist_path = os.path.join(RAGConfig.STORAGE_DIR, folder_name)
    
    if os.path.exists(persist_path):
        print(f"      [DISK] Učitavam indeks: {folder_name}")
        storage_context = StorageContext.from_defaults(persist_dir=persist_path)
        return load_index_from_storage(storage_context, embed_model=embed_model)
    
    print(f"      [BUILD] Gradim NOVI indeks: {folder_name}")
    
    if strategy_config["type"] == "fixed":
        parser = SentenceSplitter(chunk_size=strategy_config["chunk_size"], chunk_overlap=strategy_config["chunk_overlap"])
    else:
        parser = SemanticSplitterNodeParser(buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embed_model)
        
    nodes = parser.get_nodes_from_documents(documents)
    index = VectorStoreIndex(nodes=nodes, embed_model=embed_model, show_progress=True)
    index.storage_context.persist(persist_dir=persist_path)
    return index

In [33]:
def evaluate_openai_judge(question, expected, generated_text, context):
    eval_client = openai.OpenAI(api_key=RAGConfig.OPENAI_API_KEY)
    
    eval_prompt = f"""
    You are an expert evaluator. Output a JSON object.
    Question: {question}
    Expected Answer: {expected}
    Generated Answer: {generated_text}
    Context Used: {context}
    
    Task: Rate metrics from 0.0 to 1.0:
    - faithfulness: Is the generated answer derived ONLY from the context?
    - context_relevancy: Does the retrieved context contain the answer?
    - answer_correctness: Is the generated answer factually correct?
    
    JSON ONLY format: {{ "faithfulness": X.X, "context_relevancy": X.X, "answer_correctness": X.X }}
    """
    
    try:
        res = eval_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "system", "content": "You are a helpful assistant designed to output JSON."},
                      {"role": "user", "content": eval_prompt}],
            response_format={"type": "json_object"} 
        )
        return json.loads(res.choices[0].message.content)
    except Exception as e:
        print(f"Greška pri evaluaciji: {e}")
        return {"faithfulness": 0, "context_relevancy": 0, "answer_correctness": 0}

In [ ]:
def evaluate_gemini_judge(question, expected, generated_text, context):
    model = genai.GenerativeModel(RAGConfig.GEMINI_EVAL_MODEL)
    
    eval_prompt = f"""
    You are an expert evaluator. Output a JSON object.
    Question: {question}
    Expected Answer: {expected}
    Generated Answer: {generated_text}
    Context Used: {context}
    
    Task: Rate metrics from 0.0 to 1.0:
    - faithfulness: Is the generated answer derived ONLY from the context?
    - context_relevancy: Does the retrieved context contain the answer?
    - answer_correctness: Is the generated answer factually correct?
    
    JSON ONLY format: {{ "faithfulness": X.X, "context_relevancy": X.X, "answer_correctness": X.X }}
    """
    
    try:
        res = model.generate_content(
            eval_prompt,
            generation_config={"response_mime_type": "application/json"},
            request_options={"timeout": 30.0} 
        )
        return json.loads(res.text)
    except Exception as e:
        print(f"Greška pri evaluaciji (Gemini): {e}")
        return {"faithfulness": 0, "context_relevancy": 0, "answer_correctness": 0}

In [ ]:
import os
import time
import json
import pandas as pd
from datetime import datetime

pipelines = [
    {"name": "OpenAI-GPT4o-Mini", "llm": llm_openai, "embed": embed_model_openai, "suffix": "openai"},
    {"name": "Google-Gemini-Flash", "llm": llm_gemini, "embed": embed_model_gemini, "suffix": "gemini"}
]

all_results = []
backup_csv_path = f"{RAGConfig.RESULTS_DIR}/backup_multi_dataset.csv"

for dataset_name, paths in RAGConfig.DATASETS.items():
    print(f"\n{'='*50}")
    print(f" POKREĆEM OBRADU ZA: {dataset_name}")
    print(f"{'='*50}")
    
    print(f"Učitavam PDF-ove iz: {paths['pdf_dir']}")
    reader = SimpleDirectoryReader(input_dir=paths['pdf_dir'], required_exts=[".pdf"], recursive=True)
    current_documents = reader.load_data()
    
    print(f"Učitavam pitanja iz: {paths['qa_path']}")
    with open(paths['qa_path'], 'r', encoding='utf-8') as f:
        current_questions = json.load(f).get("evaluation_questions", [])
        
    print(f"-> {len(current_documents)} dokumenata, {len(current_questions)} pitanja.\n")

    for strategy_name, strategy_config in RAGConfig.CHUNKING_STRATEGIES.items():
        print(f"\n--- STRATEGIJA: {strategy_name} ---")
        
        for pipe in pipelines:
            print(f" >> Pipeline: {pipe['name']}")
            
            try:
                current_index = get_or_build_index(
                    dataset_name=dataset_name, 
                    strategy_name=strategy_name, 
                    strategy_config=strategy_config, 
                    embed_model=pipe['embed'], 
                    suffix=pipe['suffix'],
                    documents=current_documents  
                )
                
                query_engine = current_index.as_query_engine(llm=pipe['llm'], similarity_top_k=5)
                
                for i, qa in enumerate(current_questions):
                    print(f"    Pitanje {i+1}/{len(current_questions)}: {qa['question'][:40]}...", end="\r")
                    
                    nodes = current_index.as_retriever(similarity_top_k=5).retrieve(qa["question"])
                    context = "\n".join([n.text for n in nodes])
                    
                    start = time.time()
                    response = query_engine.query(qa["question"])
                    generated_answer = response.response
                    latency = time.time() - start
                    
                    oa_scores = evaluate_openai_judge(qa["question"], qa["expected_answer"], generated_answer, context)
                    ge_scores = evaluate_gemini_judge(qa["question"], qa["expected_answer"], generated_answer, context)
                    
                    all_results.append({
                        "dataset": dataset_name,
                        "strategy": strategy_name,
                        "pipeline": pipe['name'],
                        "question": qa["question"],
                        "generated_answer": generated_answer,
                        "latency": latency,
                        "oa_faithfulness": oa_scores.get("faithfulness", 0),
                        "oa_relevancy": oa_scores.get("context_relevancy", 0),
                        "oa_correctness": oa_scores.get("answer_correctness", 0),
                        "ge_faithfulness": ge_scores.get("faithfulness", 0),
                        "ge_relevancy": ge_scores.get("context_relevancy", 0),
                        "ge_correctness": ge_scores.get("answer_correctness", 0)
                    })
                    
                    if (i + 1) % 10 == 0:
                        df_backup = pd.DataFrame(all_results)
                        df_backup = df_backup.drop_duplicates(subset=["dataset", "strategy", "pipeline", "question"], keep="last")
                        df_backup.to_csv(backup_csv_path, index=False)
                    
                    time.sleep(RAGConfig.SLEEP_BETWEEN_QUERIES)
                    
            except Exception as e:
                print(f"\n[ERROR] Greška u {pipe['name']} / {dataset_name}: {e}")

df_results = pd.DataFrame(all_results)

df_results = df_results.drop_duplicates(subset=["dataset", "strategy", "pipeline", "question"], keep="last")

t = datetime.now().strftime("%Y%m%d_%H%M")
csv_path = f"{RAGConfig.RESULTS_DIR}/benchmark_multi_dataset_{t}.csv"
df_results.to_csv(csv_path, index=False)

print(f"\n\n=== OBRADA ZAVRŠENA ===")
print(f"Ukupno jedinstvenih rezultata: {len(df_results)}")
print(f"Rezultati spremljeni u: {csv_path}")


 POKREĆEM OBRADU ZA: Dataset_2
Učitavam PDF-ove iz: ./Dokumenti/EU/
Učitavam pitanja iz: ./eq_eu_treaty.json
-> 116 dokumenata, 102 pitanja.


--- STRATEGIJA: standard_overlap50 ---
 >> Pipeline: OpenAI-GPT4o-Mini
      [DISK] Učitavam indeks: Dataset_2_standard_overlap50_openai


2026-02-25 21:33:05,913 - INFO - Loading all indices.


2026-02-25 21:33:06,208 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-25 21:33:06,486 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-25 21:33:07,892 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-25 21:33:09,431 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


2026-02-25 21:33:17,169 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-25 21:33:17,476 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-25 21:33:18,442 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-25 21:33:19,911 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


2026-02-25 21:33:24,174 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-25 21:33:24,430 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-25 21:33:25,298 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-25 21:33:26,845 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

2026-02-25 21:49:26,141 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-25 21:49:26,341 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-25 21:49:27,810 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-25 21:49:30,090 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


2026-02-25 21:49:35,182 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-25 21:49:35,408 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-25 21:49:36,852 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-25 21:49:38,421 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


2026-02-25 21:49:42,275 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-25 21:49:42,487 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-25 21:49:43,924 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-25 21:49:45,527 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


2026-02-25 21:49:50,303 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-25 21:49:50,552 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2026-02-25 21:49:51,504 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-25 21:49:53,036 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


 >> Preskačem Google-Gemini-Flash za strategiju standard_overlap50 (već odrađeno).


=== OBRADA ZAVRŠENA ===
Ukupno jedinstvenih rezultata: 102
Rezultati spremljeni u: ./Rezultati/benchmark_multi_dataset_20260225_2149.csv


In [25]:
import pandas as pd

# Pretvaramo cijelu listu iz memorije u DataFrame
df_konacno = pd.DataFrame(all_results)

# Uklanjamo moguće duplikate (za svaki slučaj)
df_konacno = df_konacno.drop_duplicates(subset=["dataset", "strategy", "pipeline", "question"], keep="last")

# Definiramo putanju do tvog postojećeg backup filea
putanja_backupa = f"{RAGConfig.RESULTS_DIR}/backup_multi_dataset.csv"

# Prepisujemo stari backup najnovijim stanjem (koje uključuje i zadnjih 7-8 pitanja)
df_konacno.to_csv(putanja_backupa, index=False)

print(f"✅ SVI podaci (ukupno {len(df_konacno)} redova) su uspješno spremljeni u tvoj glavni backup!")
print(f"Ažurirana datoteka: {putanja_backupa}")

✅ SVI podaci (ukupno 395 redova) su uspješno spremljeni u tvoj glavni backup!
Ažurirana datoteka: ./Rezultati/backup_multi_dataset.csv
